In [ ]:
# !pip uninstall -y xlsr_finetune
# !pip install -Uqqq git+https://github.com/huggingface/transformers.git
# !pip install -Uqqq git+https://github.com/morganmcg1/xlsr_finetune.git

In [ ]:
%load_ext autoreload
%autoreload 2

# Train Demo

In [ ]:
from xlsr_finetune.data import *
from xlsr_finetune.training import *
from xlsr_finetune.wandbutils import *

In [ ]:
import os
import wandb
import random
import numpy as np
from functools import partial
from datasets import load_dataset, load_from_disk
from transformers import Wav2Vec2CTCTokenizer, Wav2Vec2FeatureExtractor, Wav2Vec2Processor, Wav2Vec2ForCTC
from transformers import Trainer, TrainingArguments

## Load data

In [ ]:
data_dir = '../../data'

In [ ]:
train_ds = load_dataset("common_voice", "ga-IE", split="train+validation", cache_dir=data_dir)
valid_ds = load_dataset("common_voice", "ga-IE", split="test", cache_dir=data_dir)
test_ds = valid_ds

Reusing dataset common_voice (../../data/common_voice/ga-IE/6.1.0/0041e06ab061b91d0a23234a2221e87970a19cf3a81b20901474cffffeb7869f)
Reusing dataset common_voice (../../data/common_voice/ga-IE/6.1.0/0041e06ab061b91d0a23234a2221e87970a19cf3a81b20901474cffffeb7869f)


Drop any rows where "path" doesn't contain a file

In [ ]:
# # if files have moved location between sessions, remap the path location
# def remap_data_dir(e):
#     e['path'] = f'{data_dir}/' + '/'.join(e['path'].split('/')[1:])
#     return e

# train_ds = train_ds.map(remap_data_dir)
# valid_ds = valid_ds.map(remap_data_dir)

Loading cached processed dataset at ../../data/common_voice/ga-IE/6.1.0/0041e06ab061b91d0a23234a2221e87970a19cf3a81b20901474cffffeb7869f/cache-86b23ad84e349908.arrow
Loading cached processed dataset at ../../data/common_voice/ga-IE/6.1.0/0041e06ab061b91d0a23234a2221e87970a19cf3a81b20901474cffffeb7869f/cache-d0027b289f39b354.arrow


In [ ]:
train_ds = drop_missing_files(train_ds)
valid_ds = drop_missing_files(valid_ds)

Loading cached processed dataset at ../../data/common_voice/ga-IE/6.1.0/0041e06ab061b91d0a23234a2221e87970a19cf3a81b20901474cffffeb7869f/cache-2683b0581eac87d8.arrow
Loading cached processed dataset at ../../data/common_voice/ga-IE/6.1.0/0041e06ab061b91d0a23234a2221e87970a19cf3a81b20901474cffffeb7869f/cache-f2c301a2d2b3595e.arrow
Loading cached processed dataset at ../../data/common_voice/ga-IE/6.1.0/0041e06ab061b91d0a23234a2221e87970a19cf3a81b20901474cffffeb7869f/cache-97f0418401e908ad.arrow
Loading cached processed dataset at ../../data/common_voice/ga-IE/6.1.0/0041e06ab061b91d0a23234a2221e87970a19cf3a81b20901474cffffeb7869f/cache-f8ff36573e42bd90.arrow


All files found
All files found


[Optional] Merge another Dataset to Your training dataset and shuffle them

In [ ]:
new_ds = load_dataset("common_voice", "en", split="test[20:40%]", cache_dir=data_dir)
# en_train_sample = load_dataset("common_voice", "en", split="test", cache_dir=data_dir)

Reusing dataset common_voice (../../data/common_voice/en/6.1.0/0041e06ab061b91d0a23234a2221e87970a19cf3a81b20901474cffffeb7869f)


In [ ]:
new_ds = new_ds.map(remap_data_dir)
# en_train_sample = en_train_sample.map(remap_data_dir)

Loading cached processed dataset at ../../data/common_voice/en/6.1.0/0041e06ab061b91d0a23234a2221e87970a19cf3a81b20901474cffffeb7869f/cache-769dfb14fdb4fbdb.arrow


In [ ]:
new_ds = drop_missing_files(new_ds)
train_ds = merge_ds(train_ds, new_ds)

Loading cached processed dataset at ../../data/common_voice/en/6.1.0/0041e06ab061b91d0a23234a2221e87970a19cf3a81b20901474cffffeb7869f/cache-3a6d8b4f90f0e2fe.arrow
Loading cached processed dataset at ../../data/common_voice/en/6.1.0/0041e06ab061b91d0a23234a2221e87970a19cf3a81b20901474cffffeb7869f/cache-f7ce85d3712c7519.arrow


All files found


Clean data and create Vocab

In [ ]:
train_ds = train_ds.map(remove_special_characters)
valid_ds = valid_ds.map(remove_special_characters)

Loading cached processed dataset at ../../data/common_voice/ga-IE/6.1.0/0041e06ab061b91d0a23234a2221e87970a19cf3a81b20901474cffffeb7869f/cache-94a50a5eaf5ba05b.arrow


In [ ]:
vocab = extract_vocab(train_ds, valid_ds, save=True, save_dir='data')
vocab

{'t': 0,
 'é': 1,
 'a': 2,
 'p': 3,
 's': 4,
 'x': 5,
 'í': 6,
 'g': 7,
 'r': 8,
 'j': 9,
 'u': 10,
 'h': 11,
 'y': 12,
 'i': 13,
 'b': 14,
 'q': 15,
 'c': 16,
 'z': 17,
 'w': 18,
 'ú': 19,
 "'": 20,
 'n': 21,
 '-': 22,
 'd': 23,
 'ó': 24,
 'k': 25,
 'o': 26,
 'v': 27,
 'á': 28,
 'e': 29,
 'm': 30,
 'l': 32,
 'f': 33,
 '|': 31,
 '[UNK]': 34,
 '[PAD]': 35}

Check your vocab and add additional characters to ignore to the `chars_to_ignore_regex` string if needed

In [ ]:
chars_to_ignore_regex = chars_to_ignore_regex[:-1] + '\𝓧]'
chars_to_ignore_regex

'[\\,\\?\\.\\!\\-\\;\\:"\\“\\%\\‘\\”\\�\\(\\)\\-\\*\\/\\\\\\𝓧]'

In [ ]:
new_remove_special_characters = partial(remove_special_characters, 
                                        chars_to_ignore_regex=chars_to_ignore_regex)
train_ds = train_ds.map(new_remove_special_characters)
valid_ds = valid_ds.map(new_remove_special_characters)

Loading cached processed dataset at ../../data/common_voice/ga-IE/6.1.0/0041e06ab061b91d0a23234a2221e87970a19cf3a81b20901474cffffeb7869f/cache-d83bbd1bc3f17377.arrow


In [ ]:
vocab = extract_vocab(train_ds, valid_ds, save=True, save_dir='data')
vocab

{'t': 0,
 'é': 1,
 'a': 2,
 'p': 3,
 's': 4,
 'x': 5,
 'í': 6,
 'g': 7,
 'r': 8,
 'j': 9,
 'u': 10,
 'h': 11,
 'y': 12,
 'i': 13,
 'b': 14,
 'q': 15,
 'c': 16,
 'z': 17,
 'w': 18,
 'ú': 19,
 "'": 20,
 'n': 21,
 'd': 22,
 'ó': 23,
 'k': 24,
 'o': 25,
 'v': 26,
 'á': 27,
 'e': 28,
 'm': 29,
 'l': 31,
 'f': 32,
 '|': 30,
 '[UNK]': 33,
 '[PAD]': 34}

## Convert Audio to Array

In [ ]:
sp2a = partial(speech_file_to_array, resample=True, new_sr=16_000)

In [ ]:
train_ds = train_ds.map(sp2a, num_proc=8)
valid_ds = valid_ds.map(sp2a, num_proc=8)

Loading cached processed dataset at ../../data/common_voice/ga-IE/6.1.0/0041e06ab061b91d0a23234a2221e87970a19cf3a81b20901474cffffeb7869f/cache-046b479ce551b5df.arrow
Loading cached processed dataset at ../../data/common_voice/ga-IE/6.1.0/0041e06ab061b91d0a23234a2221e87970a19cf3a81b20901474cffffeb7869f/cache-ab724829bc0fec12.arrow
Loading cached processed dataset at ../../data/common_voice/ga-IE/6.1.0/0041e06ab061b91d0a23234a2221e87970a19cf3a81b20901474cffffeb7869f/cache-b7d140b57de3c609.arrow


Loading cached processed dataset at ../../data/common_voice/ga-IE/6.1.0/0041e06ab061b91d0a23234a2221e87970a19cf3a81b20901474cffffeb7869f/cache-4b0056f882dd065e.arrow


Loading cached processed dataset at ../../data/common_voice/ga-IE/6.1.0/0041e06ab061b91d0a23234a2221e87970a19cf3a81b20901474cffffeb7869f/cache-6527056f9e1a7b60.arrow
Loading cached processed dataset at ../../data/common_voice/ga-IE/6.1.0/0041e06ab061b91d0a23234a2221e87970a19cf3a81b20901474cffffeb7869f/cache-0725e80ba8042b9a.arrow
Loading cached processed dataset at ../../data/common_voice/ga-IE/6.1.0/0041e06ab061b91d0a23234a2221e87970a19cf3a81b20901474cffffeb7869f/cache-13653087c870bd2b.arrow
Loading cached processed dataset at ../../data/common_voice/ga-IE/6.1.0/0041e06ab061b91d0a23234a2221e87970a19cf3a81b20901474cffffeb7869f/cache-a4c0835ee167930e.arrow


## Filter Out Files That Could Not Be Read

`speech_file_to_array` adds 0 to `speech` items where the path could not be read. Lets remove these

In [ ]:
prev_l = len(train_ds)
train_ds = train_ds.filter(lambda example: len(example['speech'])>1, batch_size=1)
print(f'{prev_l - len(train_ds)} samples removed')


0 samples removed


## Filter out Long Audio in Training Set
Longer audio can cause cuda oom errors, 112k frames @ 16k sample rate == 7s of audio

In [ ]:
prev_l = len(train_ds)
train_ds = train_ds.filter(lambda example: len(example['speech'])<=160_000) 
print(f'{prev_l - len(train_ds)} samples out of {prev_l} removed')


80 samples out of 4271 removed


## Quick Data Check

In [ ]:
rand_int = random.randint(0, len(train_ds)-1)

print(f"Number of train samples: {len(train_ds)}")
print("Target text:", train_ds[rand_int]["sentence"])
print("Input array shape:", np.asarray(train_ds[rand_int]["speech"]).shape)
print("Sampling rate:", train_ds[rand_int]["sampling_rate"])

Number of train samples: 4191
Target text: if we serve tea in crystal the shop is going to expand  
Input array shape: (144000,)
Sampling rate: 16000


## PreProcess to Create Model Input Values

In [ ]:
tokenizer = Wav2Vec2CTCTokenizer("data/vocab.json", unk_token="[UNK]", 
                                 pad_token="[PAD]", word_delimiter_token="|")

feature_extractor = Wav2Vec2FeatureExtractor(feature_size=1, sampling_rate=16000, 
                                          padding_value=0.0, do_normalize=True, return_attention_mask=True)

processor = Wav2Vec2Processor(feature_extractor=feature_extractor, tokenizer=tokenizer)

In [ ]:
def prepare_dataset(batch):
    # check that all files have the correct sampling rate
    assert (
        len(set(batch["sampling_rate"])) == 1
    ), f"Make sure all inputs have the same sampling rate of {processor.feature_extractor.sampling_rate}."

    batch["input_values"] = processor(batch["speech"], sampling_rate=batch["sampling_rate"][0]).input_values
    
    with processor.as_target_processor():
        batch["labels"] = processor(batch["sentence"]).input_ids
    return batch

In [ ]:
#hide_output
# n_cpus = os.cpu_count() - 2

train_ds = train_ds.map(prepare_dataset, remove_columns=train_ds.column_names, batch_size=32, batched=True)
valid_ds = valid_ds.map(prepare_dataset, remove_columns=test_ds.column_names, batch_size=32, batched=True)

## [Optional] Store Data in W&B

We will create a W&B Run. You can now log any data you'd like to W&B Artifacts, and it will be tied to this Run. When we use `Trainer` this run will also be used.

In [ ]:
import wandb
entity, project = setup_wandb(entity='wandb', project_name='xlsr-irish', log_model=True)

wandb_run = wandb.init(name='ie-en_baseline', project='xlsr-irish', entity='wandb', 
                       tags=['ie-en','baseline'], group='baseline', reinit=True)

wandb: You can find your API key in your browser here: https://wandb.ai/authorize


wandb: Paste an API key from your profile and hit enter:  ········································


wandb: Appending key for api.wandb.ai to your netrc file: /workspace/.netrc
wandb: Currently logged in as: wandb (use `wandb login --relogin` to force relogin)


In [ ]:
# Log the datasets
ds = {'train_ready_train_ds':train_ds,
     'train_ready_valid_ds':valid_ds,
     'raw_test_ds': test_ds,
#      'raw_en_train_ds': en_train_sample
     }

for name in ds.keys():
    f_path = f'../../data/{name}'
    ds[name].save_to_disk(f_path)
    artifact = wandb.Artifact(name=name, type='dataset',
                             description='My dataset',
                             metadata={'dataset_length':len(ds[name])})
    artifact.add_dir(f_path)
    wandb_run.log_artifact(artifact)

wandb: Adding directory to artifact (./../../data/train_ready_train_ds)... Done. 7.7s
wandb: Adding directory to artifact (./../../data/train_ready_valid_ds)... Done. 1.7s
wandb: Adding directory to artifact (./../../data/raw_test_ds)... Done. 0.1s


In [ ]:
# Log vocab file
vcb_f_path = 'data/vocab.json'
artifact = wandb.Artifact(name='vocab', type='vocab', 
                          description='Vocab for combined ie and en, len 36',
                          metadata={'vocab_length':len(vocab.keys())})
artifact.add_file(vcb_f_path)
wandb_run.log_artifact(artifact)

In [ ]:
# Log processor
processor_path = './data'
processor.save_pretrained(processor_path)
artifact = wandb.Artifact(name='processor', type='processor')
artifact.add_dir(processor_path)
wandb_run.log_artifact(artifact)

wandb: Adding directory to artifact (./data)... Done. 0.1s


## Load Saved Data

In [ ]:
# with wandb.init(project='xlsr-irish', entity='wandb') as run:
#     # Connect an Artifact to your run
#     train_ds_artifact = run.use_artifact('train_ready_train_ds:v0')
# #     valid_ds_artifact = run.use_artifact('raw_en_train_ds:v0')

#     # Download model weights to a folder and return the path
#     train_ds_dir = train_ds_artifact.download()
# #     valid_ds_dir = valid_ds_artifact.download()

# # Load your Hugging Face model from that folder, e.g. SequenceClassification model
# train_ds = load_from_disk(train_ds_dir)

## Prep Training

In [ ]:
data_collator = DataCollatorCTCWithPadding(processor=processor, padding=True,
                                          pad_to_multiple_of=8, pad_to_multiple_of_labels=8)

In [ ]:
model = Wav2Vec2ForCTC.from_pretrained(
    "facebook/wav2vec2-large-xlsr-53", 
    attention_dropout=0.1,
    hidden_dropout=0.1,
    feat_proj_dropout=0.0,
    mask_time_prob=0.05,
    layerdrop=0.1,
    gradient_checkpointing=True, 
    ctc_loss_reduction="mean", 
    ctc_zero_infinity=True,
    pad_token_id=processor.tokenizer.pad_token_id,
    vocab_size=len(processor.tokenizer)
)

Some weights of Wav2Vec2ForCTC were not initialized from the model checkpoint at facebook/wav2vec2-large-xlsr-53 and are newly initialized: ['lm_head.weight', 'lm_head.bias']
You should probably TRAIN this model on a down-stream task to be able to use it for predictions and inference.


Set Training arguments

In [ ]:
training_args = TrainingArguments(
  output_dir="../../data/my_xlsr",
  group_by_length=True,
  per_device_train_batch_size=8,
  per_device_eval_batch_size=16,
  gradient_accumulation_steps=4,
  evaluation_strategy="steps",
  num_train_epochs=30,
  fp16=True,
  save_steps=96,
  eval_steps=64,
  logging_steps=8,
  learning_rate=3e-4,
  warmup_steps=96,
  save_total_limit=1,
  dataloader_num_workers=16,
    
  # WANDB LOGGING: 
  report_to = 'wandb',  # enable logging to W&B
  run_name = 'ie-en_baseline_15e',   # Name your run, optional
  load_best_model_at_end = True,  # This will ensure your best model will be uploaded to W&B
  metric_for_best_model='wer',    # Load best model based on "wer", not eval loss
  greater_is_better=False,    # Define "best" wer score as the lowest score
)

Create trainer

In [ ]:
trainer = Trainer(
    model=model,
    data_collator=data_collator,
    args=training_args,
    compute_metrics=partial(compute_wer_metric, processor=processor),   # compute_wer_metric imported from xlsr_finetune
    train_dataset=train_ds,
    eval_dataset=valid_ds,
    tokenizer=processor.feature_extractor,
)

## Set up Monitoring [optional]
Log in to Weights and Biases and set your entity (username) and project name, or else use the publicly available entity and project below

In [ ]:
setup_wandb(entity='wandb', project_name='xlsr-irish', log_model=True)

wandb: WARNING Calling wandb.login() after wandb.init() has no effect.


('wandb', 'xlsr-irish')

## Train

In [ ]:
trainer.train()

# If using W&B and not doing any futher evaluation, then use wandb.finish()
# wandb.finish()  

/opt/conda/lib/python3.7/site-packages/torch/nn/modules/module.py:760: UserWarning: Using non-full backward hooks on a Module that does not return a single Tensor or a tuple of Tensors is deprecated and will be removed in future versions. This hook will be missing some of the grad_output. Please use register_full_backward_hook to get the documented behavior.
  warnings.warn("Using non-full backward hooks on a Module that does not return a "
/opt/conda/lib/python3.7/site-packages/torch/nn/modules/module.py:795: UserWarning: Using a non-full backward hook when the forward contains multiple autograd Nodes is deprecated and will be removed in future versions. This hook will be missing some grad_input. Please use register_full_backward_hook to get the documented behavior.
  warnings.warn("Using a non-full backward hook when the forward contains multiple autograd Nodes "
/opt/conda/lib/python3.7/site-packages/torch/optim/lr_scheduler.py:134: UserWarning: Detected call of `lr_scheduler.step()

Step,Training Loss,Validation Loss,Wer,Runtime,Samples Per Second
64,3.253500,3.129127,1.000000,43.566700,11.614000
128,2.994600,3.046782,1.000000,44.413200,11.393000
192,2.922000,2.945201,1.000000,43.114500,11.736000
256,2.855300,2.858851,1.000000,43.455700,11.644000
320,2.285200,2.151958,0.997951,43.955200,11.512000
384,1.702600,1.730614,1.036289,44.519900,11.366000
448,1.304600,1.430131,0.896693,44.174700,11.455000
512,1.136300,1.291219,0.892596,44.950100,11.257000
576,0.946400,1.267455,0.856014,44.606000,11.344000
640,0.924600,1.145469,0.812116,43.526700,11.625000


TrainOutput(global_step=3930, training_loss=0.6994878225805802, metrics={'train_runtime': 17816.8342, 'train_samples_per_second': 0.221, 'total_flos': 1.9599395248616518e+19, 'epoch': 30.0, 'init_mem_cpu_alloc_delta': 69259, 'init_mem_gpu_alloc_delta': 1261898752, 'init_mem_cpu_peaked_delta': 18306, 'init_mem_gpu_peaked_delta': 0, 'train_mem_cpu_alloc_delta': 0, 'train_mem_gpu_alloc_delta': 5107040768, 'train_mem_cpu_peaked_delta': 0, 'train_mem_gpu_peaked_delta': 0})

**wandb.finish** - If using W&B and not doing any futher training or evaluation, then use wandb.finish()

In [ ]:
# wandb.finish()  

## Evaluate

In [ ]:
from xlsr_finetune.evaluation import *

In [ ]:
test_ds = load_dataset("common_voice", "ga-IE", split="test", cache_dir=data_dir)
test_ds = test_ds.map(remap_data_dir)

Reusing dataset common_voice (../../data/common_voice/ga-IE/6.1.0/0041e06ab061b91d0a23234a2221e87970a19cf3a81b20901474cffffeb7869f)
Loading cached processed dataset at ../../data/common_voice/ga-IE/6.1.0/0041e06ab061b91d0a23234a2221e87970a19cf3a81b20901474cffffeb7869f/cache-d0027b289f39b354.arrow


In [ ]:
sp2a = partial(speech_file_to_array, resample=True, new_sr=16_000, evaluate=True)
test_ds = test_ds.map(sp2a)

In [ ]:
ev = partial(evaluate_xlsr, model=model, processor=processor)
result = test_ds.map(ev, batched=True, batch_size=16)

/opt/conda/lib/python3.7/site-packages/torch/utils/checkpoint.py:25: UserWarning: None of the inputs have requires_grad=True. Gradients will be None
  warnings.warn("None of the inputs have requires_grad=True. Gradients will be None")


In [ ]:
wer_true = 100 * wer_metric.compute(predictions=result["pred_strings"], references=result["sentence"])
print(f"WER: {wer_true:2f}")

WER: 81.275600


In [ ]:
import wandb
wandb_run.log({'test/wer_true': wer_true})
wandb_run.finish()

train/loss,0.1413
train/learning_rate,0.0
train/epoch,30.0
train/global_step,3930
_runtime,23309
_timestamp,1616880760
_step,553
eval/loss,1.26948
eval/wer,0.62657
eval/runtime,50.2427
eval/samples_per_second,10.071


train/loss,█▂▂▂▂▂▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁
train/learning_rate,▃███▇▇▇▇▇▇▆▆▆▆▆▅▅▅▅▅▄▄▄▄▄▄▃▃▃▃▃▂▂▂▂▂▂▁▁▁
train/epoch,▁▁▁▂▂▂▂▂▂▃▃▃▃▃▃▄▄▄▄▄▄▅▅▅▅▅▆▆▆▆▆▇▇▇▇▇▇███
train/global_step,▁▁▁▂▂▂▂▂▂▃▃▃▃▃▃▄▄▄▄▄▄▅▅▅▅▅▆▆▆▆▆▇▇▇▇▇▇███
_runtime,▁▁▁▁▂▂▂▂▂▂▃▃▃▃▃▄▄▄▄▄▄▅▅▅▅▅▆▆▆▆▆▆▇▇▇▇▇███
_timestamp,▁▁▁▁▂▂▂▂▂▂▃▃▃▃▃▄▄▄▄▄▄▅▅▅▅▅▆▆▆▆▆▆▇▇▇▇▇███
_step,▁▁▁▂▂▂▂▂▂▃▃▃▃▃▃▄▄▄▄▄▅▅▅▅▅▅▆▆▆▆▆▇▇▇▇▇▇███
eval/loss,██▇▅▂▂▂▁▁▁▁▁▁▁▁▁▁▁▁▁▂▂▁▁▂▂▂▂▁▂▂▂▂▂▂▂▂▂▂▂
eval/wer,████▆▆▄▄▄▄▄▃▃▃▃▂▂▂▂▂▂▂▂▂▂▂▂▁▁▁▁▁▁▁▁▁▁▁▁▁
eval/runtime,▁▂▁▁▂▂▁▂▂▄▂▃▃▃▃▃▃▅▃▃▄▄▅▅▃▅▄▇▇█▅▅▆▅▆▅▆▅▆▇
eval/samples_per_second,█▇██▇▇█▇▆▅▇▆▆▅▆▆▆▄▆▆▅▅▃▄▆▄▄▂▁▁▄▄▃▄▃▄▃▃▃▂


In [ ]:
# import wandb
# wandb.log({'test/wer_true': wer_true})
# wandb.finish()